In [2]:
# Configs
dataset_dir = r'C:\Users\fardin\Projects\EnhanceSEO\datasets\MaliciousURL1\malicious_phish.csv'

In [3]:
# import zlib
# file_name = '600_650_batch_ugxHSbe5cpWquoeKBxEcEPaz'
# with open(rf'TestsOnMaliciousURL1\ChatGPTData\{file_name}', 'rb') as f:
#     decompressed = zlib.decompress(f.read()).decode()

# with open(rf'TestsOnMaliciousURL1\ChatGPTData\{file_name}.jsonl', 'wt') as f:
#     f.write(decompressed)

In [4]:
import pandas as pd
import numpy as np
import torch
import json
from copy import copy, deepcopy
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import pandas as pd
from torch.utils.data import Dataset, DataLoader, TensorDataset
from transformers import BertTokenizer
from urllib.parse import urlparse
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize, TweetTokenizer
import time
import matplotlib.pyplot as plt
import random
from torch_scatter import scatter_max
from torch_scatter import scatter_mean
from torch_scatter import scatter_sum
from torch_scatter import scatter_std
import torchmetrics
import lightning as L
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint
from torch.utils.flop_counter import FlopCounterMode

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 1024
device

device(type='cuda')

In [6]:
df = pd.read_csv(dataset_dir)
df.columns=['Address', 'Topic']
df.dropna(inplace=True)
class_list = df.Topic.unique()
class_id = {t:i for i, t in enumerate(class_list)}
id_class = {i:t for i, t in enumerate(class_list)}

In [7]:
class_id

{'phishing': 0, 'benign': 1, 'defacement': 2, 'malware': 3}

In [8]:
df

,Address,Topic
0,br-icloud.com.br,phishing
1,mp3raid.com/music/krizz_kaliko.html,benign
2,bopsecrets.org/rexroth/cr/1.htm,benign
3,http://www.garage-pirenne.be/index.php?option=...,defacement
4,http://adventure-nicaragua.net/index.php?optio...,defacement
...,...,...
651186,xbox360.ign.com/objects/850/850402.html,phishing
651187,games.teamxbox.com/xbox-360/1860/Dead-Space/,phishing
651188,www.gamespot.com/xbox360/action/deadspace/,phishing
651189,en.wikipedia.org/wiki/Dead_Space_(video_game),phishing


In [9]:
# template_dict = {"custom_id": "", "method": "POST", "url": "/v1/embeddings", "body": {"model": "text-embedding-3-small", "input": "","max_tokens": 1000}}
# intervals = 50000
# previous = 0
# for i in range(intervals, len(df)+intervals, intervals):
#     i = min(len(df), i)
#     print(previous, i)
#     list_of_urls = []
#     for j in range(previous, i):
#         url = df['url'].iloc[j]
#         template_dict["custom_id"] = f"url-{j}"
#         template_dict["body"]["input"] = url
#         list_of_urls.append(json.dumps(template_dict))
        
#     with open(fr"TestsOnMaliciousURL1\ChatGPTData\samples_{previous}_{i}.jsonl", 'ta') as f:
#         for url in list_of_urls:
#             f.write(url + "\n")
#     previous = i
    

In [10]:
GPTEmbeddingPaths = [
    r'TestsOnMaliciousURL1\ChatGPTData\0_50_batch_uDZTTt9s8Cjyag0CVRuZxPtn_output.jsonl',
    r'TestsOnMaliciousURL1\ChatGPTData\50_100_batch_1iT04A6jdjUhSyrjJmk8hUEw_output.jsonl',
    r'TestsOnMaliciousURL1\ChatGPTData\100_150_batch_Lfrz2pFzhUPGmv5cItIJk1q1_output.jsonl',
    r'TestsOnMaliciousURL1\ChatGPTData\150_200_batch_QIB4ydKaFwYUTGv5R7K1C4fu_output.jsonl',
    r'TestsOnMaliciousURL1\ChatGPTData\200_250_batch_Nsc3rpUsTBVbFa8CaB5UWPXw.jsonl',
    r'TestsOnMaliciousURL1\ChatGPTData\250_300_batch_3ZTyxxMrukZomJfRlempxB0Z.jsonl',
    r'TestsOnMaliciousURL1\ChatGPTData\300_350_batch_CDGbePCcdBiqK5RsW7czOK6O.jsonl',
    r'TestsOnMaliciousURL1\ChatGPTData\350_400_batch_zk5FnhCB5JAxu4fupZTQkxbk.jsonl',
    r'TestsOnMaliciousURL1\ChatGPTData\400_450_batch_Q2o70ATqnH2yRAg4SWD8JXqk.jsonl',
    r'TestsOnMaliciousURL1\ChatGPTData\450_500_batch_4YySMvPDyyCkUjViQIafcuzd.jsonl',
    r'TestsOnMaliciousURL1\ChatGPTData\500_550_batch_CU0mEgcbOGt4ylwNhWDRNbq9.jsonl',
    r'TestsOnMaliciousURL1\ChatGPTData\550_600_batch_Jm9Pdo74WpPk4tvsTpQnDmz2.jsonl',
    r'TestsOnMaliciousURL1\ChatGPTData\600_650_batch_ugxHSbe5cpWquoeKBxEcEPaz.jsonl',
    r'TestsOnMaliciousURL1\ChatGPTData\650_651_batch_0eIufBJHsSd2vMEAJ97RYI6u_output.jsonl'
]

In [11]:
data_embeddings = {}
for emb_path in GPTEmbeddingPaths:
    with open(emb_path, 'rt') as f:
        all_lines = []
        for line in f.readlines():
            all_lines.append(json.loads(line))
        for url_data in all_lines:
            data_embeddings[int(url_data['custom_id'][4:])] = np.array(url_data['response']['body']['data'][0]['embedding'])

In [12]:
embedding_dim = len(url_data['response']['body']['data'][0]['embedding'])
embeddings = torch.zeros((len(data_embeddings), embedding_dim))
labels = torch.zeros((len(data_embeddings), len(class_id)))
for i, (k, v) in enumerate(data_embeddings.items()):
    embeddings[i] = torch.from_numpy(v)
    labels[i][class_id[df['Topic'].iloc[k]]] = 1

In [13]:
train_embs, test_embs, train_labels, test_labels = train_test_split(embeddings, labels, test_size=0.1, shuffle=True)
train_dataset = TensorDataset(train_embs, train_labels)
test_dataset = TensorDataset(test_embs, test_labels)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, drop_last=True, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, drop_last=True, shuffle=False)

In [49]:
class GPTEmbeddingClassifier(torch.nn.Module):
    
    def __init__(self, input_dim, num_classes, *args, **kwargs) -> None:
        super(GPTEmbeddingClassifier, self).__init__(*args, **kwargs)
        self.fc1 = nn.Linear(input_dim, 256)
        self.bn = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 256)
        self.fc3 = nn.Linear(256, num_classes)
        
    def forward(self, x):
        x = F.relu(self.bn(self.fc1(x)))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [50]:

class ClassifierLightningModel(L.LightningModule):
    def __init__(
        self,
        model,
        num_classes,
        optimizer=None,
        loss_func=None,
        learning_rate=0.01,
        batch_size=64,
        lr_scheduler=None,
        user_lr_scheduler=False,
        min_lr=0.0,
    ):
        super(ClassifierLightningModel, self).__init__()
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.model = model
        self.min_lr = min_lr
        # self.save_hyperparameters(ignore=["model"])
        self.save_hyperparameters("model", logger=False)
        self.optimizer = self._get_optimizer(optimizer)
        self.lr_scheduler = (
            self._get_lr_scheduler(lr_scheduler) if user_lr_scheduler else None
        )
        self.loss_func = loss_func
        self.train_losses = []
        self.val_losses = []
        self.train_acc = torchmetrics.Accuracy(task="multiclass", num_classes=num_classes)
        self.val_acc = torchmetrics.Accuracy(task="multiclass", num_classes=num_classes)
        self.test_acc = torchmetrics.Accuracy(task="multiclass", num_classes=num_classes)

    def forward(self, x, *args, **kwargs):
        return self.model(x)

    # def on_train_epoch_start(self) -> None:
    #     param_groups = next(iter(self.optimizer.param_groups))
    #     if "lr" in param_groups and param_groups["lr"] is not None:
    #         current_learning_rate = float(param_groups["lr"])
    #         self.log(
    #             "lr",
    #             current_learning_rate,
    #             batch_size=self.batch_size,
    #             on_epoch=True,
    #             on_step=False,
    #         )

    def training_step(self, batch, *args, **kwargs):
        X, y = batch
        X.to(self.device)
        y.to(self.device)
        
        self.model.train()
        y_out = self(X)

        loss = self.loss_func(y_out.view(y.shape), y )
        self.train_losses.append(loss.detach().item())
        self.log(
            "train_loss",
            loss,
            prog_bar=True,
            batch_size=self.batch_size,
            on_epoch=True,
            on_step=True,
        )
        
        self.train_acc(torch.argmax(y_out, dim=1), torch.argmax(y, dim=1))
        self.log('train_acc', self.train_acc, prog_bar=True, on_epoch=True, on_step=True, batch_size=self.batch_size)
        
        return loss

    def validation_step(self, batch, *args, **kwargs):
        X, y = batch
        X.to(self.device)
        y.to(self.device)
        
        self.model.eval()
        y_out = self(X)
        loss = self.loss_func(y_out.view(y.shape), y )
        self.val_losses.append(loss.detach().item())

        self.log(
            "val_loss",
            loss,
            prog_bar=True,
            batch_size=self.batch_size,
            on_epoch=True
        )
        
        self.val_acc(torch.argmax(y_out, dim=1), torch.argmax(y, dim=1))
        self.log('val_acc', self.val_acc, prog_bar=True, on_epoch=True, batch_size=self.batch_size)

    def configure_optimizers(self):
        if self.lr_scheduler is None:
            return self.optimizer

        return {
            "optimizer": self.optimizer,
            "lr_scheduler": {
                "scheduler": self.lr_scheduler,
                "monitor": "train_loss",
                "interval": "epoch",
                "frequency": 1,
            },
        }

    def update_learning_rate(self, learning_rate: float):
        self.learning_rate = learning_rate
        for g in self.optimizer.param_groups:
            g["lr"] = learning_rate

    def _get_optimizer(self, optimizer):
        return (
            optimizer
            if optimizer is not None
            else torch.optim.Adam(self.model.parameters(), lr=self.learning_rate)
        )

    def _get_lr_scheduler(self, lr_scheduler):
        return (
            lr_scheduler
            if lr_scheduler is not None
            else torch.optim.lr_scheduler.ReduceLROnPlateau(
                self.optimizer, patience=5, factor=0.5, mode="min", min_lr=self.min_lr
            )
        )


In [45]:
lr=0.00339
input_dim = 1536 
num_classes = len(class_id)

In [46]:
# for i in range(5):
callbacks = [
    # EarlyStopping(monitor='val_loss',mode='min',patience=25),
    # CustomModelCheckpoint(dirpath=r'models\malicious_urls_model', filename=f'malicious_urls_model_', every_n_epochs=3, mode='min', monitor='val_loss_epoch', save_on_train_epoch_end=True),
    ModelCheckpoint(save_top_k=5, mode='max', monitor='val_acc', save_last=True)
    ]

classifier_torch_model = GPTEmbeddingClassifier(embedding_dim, num_classes).to(device)
optimizer = torch.optim.Adam(classifier_torch_model.parameters(), lr=lr, weight_decay=0.00001)
lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[25, 50, 75, 100, 125, 150, 175],gamma=0.5)
loss_func = torch.nn.BCEWithLogitsLoss()
classfier_lightning_model = ClassifierLightningModel(classifier_torch_model, 
                                                    num_classes=num_classes,
                                            learning_rate=lr,
                                            batch_size=batch_size,
                                            optimizer=optimizer,
                                            loss_func=loss_func,
                                            lr_scheduler=lr_scheduler,
                                            user_lr_scheduler=False
                                            ).to(device)

from lightning.pytorch.loggers import CSVLogger
import lightning as L

trainer = L.Trainer(
            callbacks=callbacks,
            max_epochs=200,
            accelerator= 'gpu' if torch.cuda.is_available() else 'cpu',
            logger=CSVLogger(save_dir='logs/', name='log2')
        )

c:\Users\fardin\AppData\Local\Programs\Python\Python311\Lib\site-packages\lightning\pytorch\utilities\parsing.py:198: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [47]:
# from lightning.pytorch.tuner import Tuner
# tuner = Tuner(trainer)
# result = tuner.lr_find(classfier_lightning_model, train_dataloaders=train_dataloader, val_dataloaders=test_dataloader, min_lr=0.000001, max_lr=0.1, num_training=2000)
# fig = result.plot(suggest=True)
# fig.show()

In [48]:
X, y = next(iter(test_dataloader))
flopt_counter = FlopCounterMode(classfier_lightning_model.model)
with flopt_counter:
    classfier_lightning_model.model(X.to(device))

x1: torch.Size([1, 1536])
x1a: torch.Size([1, 256])
x1b: torch.Size([1, 256])
x2: torch.Size([1, 256])
x3: torch.Size([1, 256])
x4: torch.Size([1, 4])
Module                           FLOP    % Total
---------------------------  --------  ---------
 GPTEmbeddingClassifier.fc1  786.432K     85.52%
  - aten.addmm               786.432K     85.52%
GPTEmbeddingClassifier       919.552K    100.00%
 - aten.addmm                919.552K    100.00%
 GPTEmbeddingClassifier.fc2  131.072K     14.25%
  - aten.addmm               131.072K     14.25%
 GPTEmbeddingClassifier.fc3    2.048K      0.22%
  - aten.addmm                 2.048K      0.22%


In [47]:
trainer.fit(classfier_lightning_model, train_dataloaders=train_dataloader, val_dataloaders=test_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type                   | Params
-----------------------------------------------------
0 | model     | GPTEmbeddingClassifier | 460 K 
1 | loss_func | BCEWithLogitsLoss      | 0     
2 | train_acc | MulticlassAccuracy     | 0     
3 | val_acc   | MulticlassAccuracy     | 0     
4 | test_acc  | MulticlassAccuracy     | 0     
-----------------------------------------------------
460 K     Trainable params
0         Non-trainable params
460 K     Total params
1.843     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\fardin\AppData\Local\Programs\Python\Python311\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
c:\Users\fardin\AppData\Local\Programs\Python\Python311\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

c:\Users\fardin\AppData\Local\Programs\Python\Python311\Lib\site-packages\lightning\pytorch\trainer\call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


In [48]:
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from torchmetrics.classification import ConfusionMatrix
def calculate_metrics(cl_model):
    cm = ConfusionMatrix(task="multiclass", num_classes=len(class_id))

    y_pred = []
    y_true = []

    cl_model = cl_model.eval()
    cl_model.to(device)
    for X, y in tqdm(test_dataloader):
        X = X.to(device)
        with torch.no_grad():
            y_p = cl_model(X)
            y_p = y_p.cpu()
        y_pred.append(y_p)
        y_true.append(y)
    y_pred = torch.cat(y_pred, dim=0)
    y_true = torch.cat(y_true, dim=0)
    y_pred2 = torch.argmax(y_pred, dim=1)
    y_true2 = torch.argmax(y_true, dim=1)
    print(f'classification report: \n {classification_report(y_true2, y_pred2, digits=4)}')
    print(f'confusion matrix:\n {cm(y_pred2, y_true2)}')
    print('================================')


In [49]:
trainer.checkpoint_callback.best_model_path

'logs/log2\\version_84\\checkpoints\\epoch=52-step=30316.ckpt'

In [50]:
lightning_model = ClassifierLightningModel.load_from_checkpoint(rf'{trainer.checkpoint_callback.best_model_path}', map_location=None, hparams_file=None, strict=True, num_classes=num_classes).eval()

In [51]:
# classfier_lightning_model.model = classfier_lightning_model.model.eval()
# classfier_lightning_model = classfier_lightning_model.eval()
calculate_metrics(lightning_model)

100%|██████████| 63/63 [00:01<00:00, 41.39it/s]


classification report: 
               precision    recall  f1-score   support

           0     0.9510    0.9337    0.9423      9301
           1     0.9879    0.9921    0.9900     42540
           2     0.9918    0.9937    0.9928      9404
           3     0.9725    0.9630    0.9677      3267

    accuracy                         0.9825     64512
   macro avg     0.9758    0.9706    0.9732     64512
weighted avg     0.9824    0.9825    0.9824     64512

confusion matrix:
 tensor([[ 8684,   484,    51,    82],
        [  314, 42206,    16,     4],
        [   36,    20,  9345,     3],
        [   97,    14,    10,  3146]])


In [1]:
p = 0.9758
r = 0.9706
(2*p*r)/(p+r)

0.9731930538429922